# Conversational agent

In [ ]:
import os, datetime, requests, param
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import before_model
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from typing import Any

_ = load_dotenv()  # Load .env file

In [2]:
if not os.getenv("GOOGLE_API_KEY"):
    print("WARNING: GOOGLE_API_KEY not found in environment variables.")

In [3]:
# tools the model can use

import wikipedia
@tool
def get_current_temperature(latitude: float, longitude: float) -> str:
    """Fetch current temperature for given coordinates."""

    print("using tool get_current_temperature")
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'



@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia for the given query and return a short summary."""
    
    print("using tool search_wikipedia")

    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (wikipedia.exceptions.PageError, wikipedia.exceptions.DisambiguationError):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

@tool
def create_your_own(query: str) -> str:
    """This function reverses the input string."""
    print("using  create_your_own")
    return query[::-1]

from pydantic import BaseModel, Field
from typing import Literal



In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

# Use a Pydantic schema so the LLM gets a clear, validated structure
# for calling this tool (typed fields, defaults, and descriptions).

class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )

@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    print("using get_weather")
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

In [5]:
# changing the behaviour of the model, they run everytime the model runs

@before_model
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Keep only the last 5 messages in memory (user + assistant + tools)."""
    messages = state["messages"]
    print("=== MESSAGES BEFORE TRIM ===")
    for i, m in enumerate(messages):
        print(i, m.__class__.__name__, ":", getattr(m, "content", str(m)))

    if len(messages) <= 5:
        return None

    new_messages = messages[-5:]

    print("=== MESSAGES AFTER TRIM ===")
    for i, m in enumerate(new_messages):
        print(i, m.__class__.__name__, ":", getattr(m, "content", str(m)))

    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages,
        ]
    }

In [6]:
tools = [get_current_temperature, search_wikipedia, create_your_own, get_weather]

In [7]:
# agent class definition

class AgentBackend(param.Parameterized):
    def __init__(self, tools, **params):
        super().__init__(**params)

        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite",
            temperature=0,
        )

        self.memory = InMemorySaver()
        

        self.agent = create_agent(
            model=self.llm,
            tools=tools,
            middleware=[trim_messages],
            system_prompt=(
                "You are a helpful assistant. "
                "Adjust values to format them as tools required them"
                "When the user says 'this is important, remember it: ...', "
                "call the remember_fact tool."
            ),
            checkpointer=self.memory,
        )

        self.config = {"configurable": {"thread_id": "1"}}

    def run_agent(self, query: str) -> str:
        if not query:
            return ""
        result = self.agent.invoke(
            {"messages": [("user", query)]},
            config=self.config,
        )
        return result["messages"][-1].content


In [8]:
backend = AgentBackend(tools)

In [9]:
import gradio as gr

def chat_wrapper(message, history):
    return backend.run_agent(message)

demo = gr.ChatInterface(
    fn=chat_wrapper,
    title="Gemini QnA Bot",
    description="Ask me about the weather, Wikipedia, or reverse a string!",
    examples=["What is the weather in Tokyo?", "Reverse the word 'LangChain'"],
)

/Users/ema/Documents/UPM/LangChainProject/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


=== MESSAGES BEFORE TRIM ===
0 HumanMessage : what's the weather in osaka? use get_weather
using get_weather
=== MESSAGES BEFORE TRIM ===
0 HumanMessage : what's the weather in osaka? use get_weather
1 AIMessage : 
2 ToolMessage : Current weather in Osaka: 22 degrees C
=== MESSAGES BEFORE TRIM ===
0 HumanMessage : what's the weather in osaka? use get_weather
1 AIMessage : 
2 ToolMessage : Current weather in Osaka: 22 degrees C
3 AIMessage : The current weather in Osaka is 22 degrees Celsius.
4 HumanMessage : what's in tokyo?
using tool search_wikipedia
=== MESSAGES BEFORE TRIM ===
0 HumanMessage : what's the weather in osaka? use get_weather
1 AIMessage : 
2 ToolMessage : Current weather in Osaka: 22 degrees C
3 AIMessage : The current weather in Osaka is 22 degrees Celsius.
4 HumanMessage : what's in tokyo?
5 AIMessage : 
6 ToolMessage : Page: Tokyo
Summary: Tokyo, officially the Tokyo Metropolis, is the capital and most populous city in Japan. With a population of over 14 million in 